In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import os
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import numpy as np
import pydicom


def CalValidFileNum(filedir, filelist, ValidFileList):
    res = 0
    file_size = np.array([2, 1024, 256])
    for item in filelist:
        if(item.endswith('.dcm')):
            file_name = filedir + '/' + item
            file_stats = os.stat(file_name)
            if((file_stats.st_size/1024) > 1000):
                I = pydicom.read_file(file_name)
                size = I.pixel_array.shape
                if((size == file_size).all()):
                    ValidFileList.append(file_name)
                    res += 1
    return res


def norm_array(array):
    size = array.shape
    max0 = 0
    for i in range(size[0]):
        for j in range(size[1]):
            if(array[i][j] > max0):
                max0 = array[i][j]
    res = np.zeros(size)
    if(max0 > 0):
        for i in range(size[0]):
            for j in range(size[1]):
                res[i][j] = array[i][j]/max0
    return res


def ReadValidDcmFile(filelist, X, Y):
    i = 0
    for item in filelist:
        I = pydicom.read_file(item)
        Data = I.pixel_array
        temp = Data[0][:][:].astype(np.float)
        X[i, :, 0:256] = temp
        temp = Data[1][:][:].astype(np.float)
        X[i, :, 320:576] = temp
        #X[i, :, :] = norm_array(X[i, :, :])
        # p1 = X[i,:,:]
        # cv2.imshow("123",p1)
        # cv2.waitKey()
        if(I.PatientSex == 'F'):
            Y[i] = 0
        elif(I.PatientSex == 'M'):
            Y[i] = 1
        i += 1


def get_dataset(num_classes):
    filedir = './train'
    filelist = os.listdir(filedir)
    ValidFileNum = 0
    ValidFileList = list()
    ValidFileNum = CalValidFileNum(filedir, filelist, ValidFileList)
    #ValidFileNum=600
    x_train = np.zeros((ValidFileNum, 1024, 576))
    y_train = np.zeros(ValidFileNum)
    ReadValidDcmFile(ValidFileList, x_train, y_train)
    filedir = './test'
    filelist = os.listdir(filedir)
    ValidFileNum = 0
    ValidFileList = list()
    ValidFileNum = CalValidFileNum(filedir, filelist, ValidFileList)
    x_test = np.zeros((ValidFileNum, 1024, 576))
    y_test = np.zeros(ValidFileNum)
    ReadValidDcmFile(ValidFileList, x_test, y_test)
    x_train = x_train.reshape(x_train.shape[0], 1024, 576, 1)
    x_test = x_test.reshape(x_test.shape[0], 1024, 576, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test)


In [4]:
def get_model(num_classes):
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                          activation='relu',
                          padding='same',
                          input_shape=(1024, 576, 1)))
  model.add(layers.MaxPooling2D(pool_size=(4, 4)))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(4, 4)))
  model.add(layers.BatchNormalization())
  # model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
  # model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
#   model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
#   model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#   model.add(layers.BatchNormalization())
#   model.add(layers.Dropout(0.4))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))
  model.summary()
  return model


In [5]:
batch_size = 16
epochs = 50
num_classes = 2


def train(learning_rate=1.0):
  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes)
  model = get_model(num_classes)

  # Specify the optimizer (Adadelta in this example), using the learning rate input parameter of the function so that Horovod can adjust the learning rate during training
  optimizer = keras.optimizers.Adadelta(lr=learning_rate)

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=2,
            validation_data=(x_test, y_test))
  return model


model = train(learning_rate=0.02)
(x_train, y_train), (x_test, y_test) = get_dataset(num_classes)
print(sum(y_train))
loss, accuracy = model.evaluate(x_test, y_test, batch_size=80)
print("loss:", loss)
print("accuracy:", accuracy)
print('finish')


C:\Users\hp\AppData\Local\Temp\ipykernel_29992\1628942314.py:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  temp = Data[0][:][:].astype(np.float)
C:\Users\hp\AppData\Local\Temp\ipykernel_29992\1628942314.py:49: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  temp = Data[1][:][:].astype(np.float)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1024, 576, 32)     320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 256, 144, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 144, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 144, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 36, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 36, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 36, 64)       

In [6]:
model.summary()
model.save("cnn-sex.h5")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1024, 576, 32)     320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 256, 144, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 144, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 144, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 36, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 36, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 36, 64)       